In [0]:
"""
Script with all imported "helper"-functions used in the Training Session.
"""

import os
import json
import skimage
import tensorflow as tf
import numpy as np
import matplotlib.image as mpimg
from skimage.io import imread
from pathlib import Path
from collections import Counter
from math import radians, cos, sin, asin, sqrt

In [0]:
# model output shape transformation

In [0]:
def convert_to_labelmatrices(result, number_of_labels):
    """
    creates a seperate matrix for each label
    overrides overlapping labels within the same class by confidence Note: result['masks'] is boolean matrix!
    Returns X: nparray of shape (input_shape,input_shape,number_of_labels)
    """
    X = np.zeros([result['masks'].shape[0], result['masks'].shape[1], number_of_labels])
    for i, class_id in enumerate(result['class_ids']):
        # take elementwise maximum of previous matrix and new object
        X[:, :, class_id] = np.maximum(X[:, :, class_id], result['masks'][:, :, i] * (result['scores'][i]))
    return X


def convert_to_one_labelmatrix(X, labels):
    """
    converts labelmatrices to one matrix.
    Args:
        X (nparray): labelmatrix of shape (img_dim,img_dim,number of labels)
        labels (list): list of labels ordered by expected index
    Returns:
        Y (nparray): matrix of shape (img_dim,img_dim) containing label_index of each pixel
    """
    labels_without_BG = labels[1:]

    Y = np.zeros([X.shape[0], X.shape[1]])
    for label in labels_without_BG:
        for x in range(X.shape[0]):
            for y in range(X.shape[1]):
                if X[x, y, labels_without_BG.index(label)] != 0:
                    Y[x, y] = labels_without_BG.index(label)
    
    return Y

In [0]:
# Plotting

In [0]:
def display_masks_imgs(path1, path2):
    img_photo_1 = mpimg.imread(path1)
    img_photo_2 = mpimg.imread(path2)
    img_mask_1 = mpimg.imread(path1.replace('jpg', 'png'))
    img_mask_2 = mpimg.imread(path2.replace('jpg', 'png'))

    # creating multiple subplots
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(13, 13))

    # title for the entire figure
    fig.suptitle('Pictures and corresponding Masks', fontsize=20)

    # filling subplots with pictures
    axes[0, 0].imshow(img_photo_1)
    axes[0, 1].imshow(img_photo_2)
    axes[1, 0].imshow(img_mask_1)
    axes[1, 1].imshow(img_mask_2)

    # removing axes of the subplots
    axes[0, 0].axis('off')
    axes[0, 1].axis('off')
    axes[1, 0].axis('off')
    axes[1, 1].axis('off')

    return plt.show()

  
def count_label_occurrences_annotations_file(json_file):
    """

    :return:
    """
    with open(json_file, 'r') as json_f:
        label_json = json.load(json_f)

    label_counts = Counter({})
    for instance in label_json["annotations"]:
        if instance["category_id"] not in label_counts.keys():
            label_counts[instance["category_id"]] = 0
        else:
            label_counts[instance["category_id"]] += 1

    category_id_map = {}
    for category in label_json["categories"]:
        category_id_map[category["id"]] = category["name"]

    for key in category_id_map:
        if key in label_counts.keys():
            label_counts[category_id_map[key]] = label_counts.pop(key)

    return dict(label_counts.most_common())
  
  
def label_distribution_plot(path):
  train_l = count_label_occurrences_annotations_file(path)

  train_l = {key: value for (key, value) in train_l.items()}
  train_l = dict(Counter(train_l).most_common(10))

  plt.figure(figsize=(24, 8))
  plt.bar(range(len(train_l)), train_l.values(), align='center', color ='#12ABDB')
  plt.xticks(range(len(train_l)), list(train_l.keys()))
  plt.xlabel('labels')
  plt.ylabel('occurrence')
  plt.title('The occurrence of the different labels')
  plt.show()
  
  
def simple_illustrate(result, class_names):
    img = skimage.io.imread(result['Img'])
    display_instances(img, result['rois'], result['masks'], result['class_ids'], class_names, result['scores'], figsize=(16, 16))

    print('Categories: ', result['class_ids'])
    print('Categories: ', result['scores'])
    print('Mask shape: ', result['masks'].shape)

In [0]:
# PV/Biodiversity/CO2 Score Calculation Functions for predited Image

In [0]:
def image_location(image_path):
    # transforms one image path to a 2 deminsonal of longitude and latitude
    return [float(x) for x in Path(image_path).stem.split('_')]

def distance_in_km(first_location, second_location):
    # returns the distance in km between two sets of 2d arrays with longitudes and latitudes
    first_location_lat, first_location_long, second_location_lat, second_location_long = map(radians, [first_location[0], first_location[1], second_location[0], second_location[1]])
    dlon = second_location_long - first_location_long 
    dlat = second_location_lat - first_location_lat 
    
    a = sin(dlat/2)**2 + cos(first_location_lat) * cos(second_location_lat) * sin(dlon/2)**2
    distance = 6371 * (2 * asin(sqrt(a)))
    return distance


def sun_hours_nearest_station(image_location,weather_data_list):
    # returns the average yearyl sun hours from the closest weather station to the current image from a weather_data list
    nearest_weather_station = (min(weather_data_list, key=lambda weather_station: distance_in_km(image_location, weather_station)))
    return nearest_weather_station[4]


def count_distinct_value_occurances(one_label_matrix):
    occurances = np.zeros(19)
    
    # each label is counted how often it occurs in the matrix
    for x in range(one_label_matrix.shape[0]):
      for y in range(one_label_matrix.shape[1]):
           if one_label_matrix[x,y] != 0:
                occurances[int(one_label_matrix[x,y])-1] += 1
    
    return occurances

  
def apply_scores(label_pixel_count,score_dict):
    # calculate the total score as a product of pixelcount * labelscore
    score = 0
    for counter, label in enumerate(score_dict):
      score = label_pixel_count[counter] * score_dict[label] + score
    
    return score

  
def score_calculation(one_image_result,weather_data_list,label_scores):
    '''
    updates a dict including (one_image_result) with the Pv, Biodiversity and Co2 Score
        one_image_result: dict including predicted mask of an image
        label_score_list: dict with labels as keys and scores as values
    Returns: input dict updated with 3 key-values pairs of scores needed for submission
    '''
    label_scores_indexed = label_scores.set_index('labels')
    label_score_dict = label_scores_indexed.to_dict()
    # find based on the location of the image the nearest weather station
    hours_of_sun = sun_hours_nearest_station(image_location(one_image_result['Img']),weather_data_list)
    
    # count how many pixels each label has
    occurance_array = count_distinct_value_occurances(one_image_result['predicted_mask'])

    # updating the image object
    one_image_result.update({
        'emmission_score': apply_scores(occurance_array,label_score_dict['CO2']),
        'solar_score': apply_scores(occurance_array,label_score_dict['usable_area'])/25 * hours_of_sun, # 1pixel ~ 0.2m*0.2m 
        'biodiversity_score': apply_scores(occurance_array,label_score_dict['biodiversity']) 
    })
    
    return one_image_result